# HW 2 - Разложение матриц градиентным методом

Цель задания: В ходе реализации [разложения Таккера](https://proceedings.neurips.cc/paper/2018/file/45a766fa266ea2ebeb6680fa139d2a3d-Paper.pdf) градиентным методом освоить pyTorch и реализовать подходы оптимизации параметров модели (в отсутствии готовых решений).

In [2]:
# !pip install tensorly

[Более-менее внятное описание алгоритма канонического разложения](https://www.alexejgossmann.com/tensor_decomposition_tucker/) - само аналитическое разложение вам реализовывать НЕ НУЖНО

In [3]:
import random
import time
import torch
import pandas as pd
import numpy as np

import tensorly as tl
from tensorly.decomposition import tucker
from tensorly.metrics.regression import RMSE

import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot as plt
from numpy.linalg import svd, matrix_rank, pinv, inv
from scipy.linalg import eigh, eig
from sklearn.metrics import mean_squared_error
from tqdm.notebook import tqdm
from torch import nn

import math
from torch.optim.optimizer import Optimizer
import torch.nn.functional as F

torch.manual_seed(0)

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cpu


## 1 Создайте 3х мерный тензор
Размер тензора не меньше 100 по каждой из размерностей.

Заполните случайными целыми числами в диапазоне от 0 до 9.

Примечание: разложение будет корректно работать со случайным тензором, только если изначально создавать случайные ядро и матрицы, а потом по ним формировать тензор. Работайте с типом *torch.Tensor.double*.

In [54]:
# # Функция, восстанавливающая тензор по ядру и матрицам
# def repair_tensor(G_, U):
#     # data - восстановленный тензор из матриц и ядра
#     # U - список матриц
#     # G_ - ядро разложения
#     a1 = tl.tenalg.mode_dot(tensor=tl.tensor(G_.detach().numpy()), matrix_or_vector=tl.tensor(U[0].detach().numpy()), mode=0, transpose=False)
#     a2 = tl.tenalg.mode_dot(tensor=a1, matrix_or_vector=tl.tensor(U[1].detach().numpy()), mode=1, transpose=False)
#     a3 = tl.tenalg.mode_dot(tensor=a2, matrix_or_vector=tl.tensor(U[2].detach().numpy()), mode=2, transpose=False)
#     return torch.tensor(a3, dtype=torch.double, requires_grad=True, device=device)

In [69]:
# Создадим тензор: размер тензора и r задаётся
def get_tensor(size=(100,200,150), r=10):
    # data - тензор с заданной размерностью
    # U - список матриц
    U = [torch.randn(size[i], r, dtype=torch.double) for i in range(len(size))]
    # G - ядро разложения
    G = torch.randint(0, 10, (r, r, r), dtype=torch.double)
    # data = rebuild_tensor(G, U)
    ata = torch.einsum('ia,jb,kc,abc->ijk', U[0], U[1], U[2], G)

    return data, U, G

In [56]:
def rebuild_tensor(G_, U):
    result = G_
    for i, u in enumerate(U):
        result = torch.tensordot(result, u, dims=([0], [1]))
    # print(f"result tensor requires_grad: {result.requires_grad}")
    return result

Сгенерируйте тензор и добавьте к нему случайный шум с размерностью *1e-2*

In [57]:
size = (100, 200, 300)
r = 10

data, U, G = get_tensor(size, r)
data.shape, [u.shape for u in U], G.shape

NameError: name 'U1' is not defined

In [ ]:
# noise = torch.tensor(np.random.normal(0, 1e-2, size=size), dtype=torch.double, device=device)
# data_w_noise = data + noise
# print(f"data_w_noise requires_grad: {data_w_noise.requires_grad}")

In [58]:
data_w_noise = data + torch.randn_like(data) * 1e-2

print(f"data_w_noise requires_grad: {data_w_noise.requires_grad}")

data_w_noise requires_grad: False


Вопрос:
Почему задание не имеет смысла для полностью случайного тензора и зачем добавлять шум? *не отвечать нельзя*

Ответ:

## 2 Реализуйте метод для восстановления тензора по разложению

In [59]:
# Функция, восстанавливающая тензор по ядру и матрицам (для результата из библы tensorly)
def repair_tensor_for_ndarray(G_, U):
    # data - восстановленный тензор из матриц и ядра
    # U - список матриц
    # G_ - ядро разложения
    a1 = tl.tenalg.mode_dot(tensor=tl.tensor(G_), matrix_or_vector=tl.tensor(U[0]), mode=0, transpose=False)
    a2 = tl.tenalg.mode_dot(tensor=a1, matrix_or_vector=tl.tensor(U[1]), mode=1, transpose=False)
    a3 = tl.tenalg.mode_dot(tensor=a2, matrix_or_vector=tl.tensor(U[2]), mode=2, transpose=False)
    return torch.tensor(a3, dtype=torch.double)

## 3 Сделайте разложение библиотечным методом
Пакет можете брать любой

In [60]:
from tensorly.decomposition import tucker
from tensorly import tucker_to_tensor

# использую tucker from tensorly
# data_ndarray = data_w_noise.detach().cpu().numpy()
data_ndarray = data_w_noise.detach().numpy()
core, factors = tucker(tl.tensor(data_ndarray), [r, r, r])
print(core.shape, [u.shape for u in factors])

(10, 10, 10) [(100, 10), (200, 10), (150, 10)]


Не забудьте померить ошибку разложения по метрике MSE

In [61]:
def MSE(tensor1, tensor2):
    delta = tensor1-tensor2
    delta *= delta
    mse = delta.sum() / delta.numel()
    return mse.item()

In [62]:
repaired_data = repair_tensor_for_ndarray(core, factors)

In [63]:
MSE(repaired_data, data_w_noise)

9.993324875311374e-05

## 4 Реализуйте разложение градиентным методом

### 4.1 Реализуйте *optimizer*
Можно взять из исходников *PyTorch* и отнаследоваться от *torch.optim.optimizer*.
Используйте квадратичный *Loss*.

In [64]:
# class Opt(Optimizer):

#     def __init__(self, params, lr=1e-3, ...):
#         super().__init__(params, defaults)

#     def step(self):
#         return loss

### 4.2 Реализуйте цикл оптимизации параметров

Стоит параметры оптимизировать сразу на GPU

In [65]:
class TuckerOptimizer(Optimizer):
    def __init__(self, params, lr=1e-3):
        defaults = {'lr': lr}
        super().__init__(params, defaults)

    # по мотивам доков sgd optimizer
    def step(self):
        for group in self.param_groups:
            for param in group['params']:
                if param.grad is not None:
                    param.data -= group['lr'] * param.grad.data

In [66]:
def train(data, rank, lr=1e-3, epochs=1000):
    history = []

    factors = [torch.randn(data.size(i), rank[i], requires_grad=True, dtype=torch.double) for i in range(3)]
    core = torch.randn(rank, requires_grad=True, dtype=torch.double)

    # optimizer = TuckerOptimizer([{'params': factors + [core]}], lr=lr)
    optimizer = TuckerOptimizer([core] + factors, lr=1e-3)
    loss_fn = torch.nn.MSELoss()

    for epoch in range(epochs):
        optimizer.zero_grad()

        approx_tensor = rebuild_tensor(core_tensor, factors)
        loss = loss_fn(approx_tensor, data)
        loss.backward()

        optimizer.step()

        if epoch % 100 == 0:
            history.append(loss.item())
            print(f'Epoch {epoch + 1}, Loss: {loss.item()}')

    return core, factors, history


In [67]:
tensor0 = data_w_noise  # Исходный тензор

core, fac, history = train(tensor0, [10,10,10], lr=1e-9, epochs=2000)

result = rebuild_tensor(core, fac)

mse = torch.nn.functional.mse_loss(result, tensor0)
print(f"MSE после градиентного разложения: {mse.item():.6e}")
plt.plot(np.arange(20), history)

Epoch 1, Loss: 165878917.5611736
Epoch 101, Loss: nan


KeyboardInterrupt: 

In [12]:
tensor0 = data_w_noise  # Исходный тензор

rank = [10,10,10]
factors = [torch.randn(data.size(i), rank[i], requires_grad=True, dtype=torch.double) for i in range(3)]
core_tensor = torch.randn(rank, requires_grad=True, dtype=torch.double)


# optimizer = TuckerOptimizer([core_tensor] + factors, lr=1e-3)
# optimizer = torch.optim.Adam([core_tensor] + factors, lr=1e-3)
optimizer = TuckerOptimizer([{'params': factors + [core_tensor]}], lr=1e-3)
# loss_fn = torch.nn.MSELoss()
loss_fn = torch.nn.functional.mse_loss

history = []
for epoch in range(10000):
    optimizer.zero_grad()

    approx_tensor = rebuild_tensor(core_tensor, factors)
    # approx_tensor = torch.einsum('ia,jb,kc,abc->ijk', factors[0], factors[1], factors[2], core_tensor)

    loss = loss_fn(approx_tensor, tensor0)
    loss.backward()

    optimizer.step()

    if epoch % 100 == 0:
        history.append(loss.item())
        print(f'Epoch {epoch + 1}, Loss: {loss.item()}')

plt.plot(np.arange(100), history)

<class 'torch.Tensor'>
tensor0 requires_grad: False
Epoch 1, Loss: 32739.172087994415
Epoch 101, Loss: 32209.82872123223
Epoch 201, Loss: 32050.68718579427
Epoch 301, Loss: 31975.143360694095
Epoch 401, Loss: 31928.380850725087
Epoch 501, Loss: 31887.9435883454
Epoch 601, Loss: 31818.782886890443
Epoch 701, Loss: 31331.693547811832
Epoch 801, Loss: 8113.2035991015155
Epoch 901, Loss: 7977.613937121839
Epoch 1001, Loss: 7958.483963265756
Epoch 1101, Loss: 7946.863332330446
Epoch 1201, Loss: 7938.284538723272
Epoch 1301, Loss: 7930.922286865917
Epoch 1401, Loss: 7923.760913683998
Epoch 1501, Loss: 7916.024064234158
Epoch 1601, Loss: 7906.922288113009
Epoch 1701, Loss: 7895.484912380404
Epoch 1801, Loss: 7880.406692716654
Epoch 1901, Loss: 7859.905093222146
Epoch 2001, Loss: 7831.648513946093
Epoch 2101, Loss: 7792.885952067904
Epoch 2201, Loss: 7740.874834124008
Epoch 2301, Loss: 7673.418267296528
Epoch 2401, Loss: 7589.215995967727
Epoch 2501, Loss: 7488.684859363673
Epoch 2601, Loss: 7

KeyboardInterrupt: 

## 5 Приведите сравнение скорости работы и ошибки восстановления методом из пакета и реализованного градиентного
Сравнение может считаться ± объективным с размером выборки от 10.

In [71]:
import time

results = {'pocket': {'execution_time': [],
                      'mse_values': []},
           'gradient': {'execution_time': [],
                        'mse_values': []}}

sample_count = 10
tensor_shape = (50, 100, 150)
tensor_rank = 10

noisy_tensors = []
for _ in range(sample_count):
    tensor_data, _, _ = get_tensor(size=tensor_shape, r=tensor_rank)
    # шумим
    noisy_tensors.append(tensor_data + torch.randn_like(tensor_data) * 1e-2)

# градиент
for tensor in noisy_tensors:
    start_time = time.time()
    core_tensor, factors, _ = train(tensor, rank=[tensor_rank]*3, lr=1e-3, epochs=2000)
    approx_tensor = rebuild_tensor(core_tensor, factors)

    mse_value = torch.nn.functional.mse_loss(approx_tensor, tensor).item()
    duration = time.time() - start_time

    results['gradient']['execution_time'].append(duration)
    results['gradient']['mse_values'].append(mse_value)

# пакет
for tensor in noisy_tensors:
    start_time = time.time()
    core_tensor, factors = tucker(tl.tensor(tensor), rank=[tensor_rank]*3)
    core_tensor = torch.tensor(core_tensor, dtype=torch.double)
    factors = [torch.tensor(factor, dtype=torch.double) for factor in factors]

    reconstructed_tensor = rebuild_tensor(core_tensor, factors)

    # MSE
    mse_value = torch.nn.functional.mse_loss(reconstructed_tensor, tensor).item()
    duration = time.time() - start_time

    results['pocket']['execution_time'].append(duration)
    results['pocket']['mse_values'].append(mse_value)


for method in results:
    average_time = sum(results[method]['execution_time']) / sample_count
    print(f" Метод: {method}")
    print(f" Среднее время выполнения: {average_time:.6f} сек")
    average_mse = sum(results[method]['mse_values']) / sample_count
    print(f" Средняя ошибка восстановления (MSE): {average_mse:.6f}")

Epoch 1, Loss: 178215676.31965435
Epoch 101, Loss: nan


KeyboardInterrupt: 